In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from multi_env import make_reversi_vec_env, SelfPlayEnv
import torch as th
from players import RandomPlayer
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import numpy as np

In [4]:
board_shape = 8
n_envs = 10
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={
        'board_shape': board_shape,
        'local_player_cls': RandomPlayer
    }
)

# Modificación de librería para que haga argmax solo sobre las válidas

In [5]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
)

In [6]:
model.predict(env.reset())

(array([40, 13, 18, 26, 20, 26,  9, 58, 38, 47]), None)

# Custom ActorCriticPolicy 

https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/policies.py

In [7]:
from boardgame2 import ReversiEnv

In [8]:
env_not_vect = ReversiEnv(board_shape)

In [9]:
(board, player) = env_not_vect.reset()

In [10]:
env_not_vect.get_valid((board, player))

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [11]:
def get_actions_mask(state):
    player = 1
    valid_actions = env_not_vect.get_valid((state, player))
    return valid_actions.reshape(-1)  


In [12]:
get_actions_mask(env.reset()[0][0])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

## Importante: Aclaración para docentes
Reemplacé esta sección de la notebook por una implementación en archivos .py para trabajarlo con un IDE.
El enfoque que seguí fue el de utilizar un 2do canal para contener las máscaras, en vez de llamar a get_actions_mask en forward, predict, etc.
Además utilizo SubprocVecEnv (en reemplazo de DummyVecEnv), que paraleliza el rollout de episodios en multiples procesos.
Esto me permitió acelerar bastante el entrenamiento ya que las máscaras son calculadas en cada proceso y todo es convertido a tensores antes de entrar a la policy. Luego todo se ejecuta en GPU.

Mirar los siguientes archivos:
* custom_features_extractor.py => Contiene un features extractor (CustomBoardExtractor) que toma el board del canal 0 y hace un flatten, ignorando el canal 1 (mask), para alimentar las nn de actor-critic. También contiene uno para extraer features utilizando CNN (CNNFeaturesExtractor).
* custom_policies.py => Contiene la implementación de CustomActorCritic modificada, donde se utiliza el canal 1 para aplicar la máscara de acciones válidas.
* multi_env.py => Hice algunas modificaciones minimas a SelfPlayEnv para parametrizar el uso o no del canal de máscara, y además agregué poder entrenar contra distintos localPlayer en cada entorno paralelo. Esto me permite aprender de varias estrategias al mismo tiempo entre iteración e iteración, para evitar encontrar mínimos locales.
* reversi_model.py => Contiene una clase de alto nivel (CustomReversiModel) para configurar, entrenar y guardar el modelo. Se utiliza en esta notebook.
* learn.py => Script de entrada para entrenar por línea de comandos. Contiene múltiples argumentos que permiten customizar y rapidamente realizar un entrenamiento, creando un modelo nuevo o cargando uno preexistente, compitiendo contra una o múltiples estrategias.

In [13]:
from reversi_model import CustomReversiModel
from multi_env import SelfPlayEnv
from players import RandomPlayer
import torch as th
import numpy as np

In [14]:
board_shape = 8
n_envs = 8
gamma = 0.99
ent_coef = 0.001
gae_lambda = 0.9
n_epochs = 10
n_steps = 2048
learning_rate = 2e-4
batch_size = 64

In [15]:
model = CustomReversiModel(board_shape=board_shape,
                           n_envs=n_envs,
                           local_player=RandomPlayer,
                           n_steps=n_steps,
                           n_epochs=n_epochs,
                           learning_rate=learning_rate,
                           ent_coef=ent_coef,
                           gae_lambda=gae_lambda,
                           batch_size=batch_size,
                           verbose=True)

Using cuda device


In [16]:
env = SelfPlayEnv(board_shape=board_shape, local_player_cls=RandomPlayer, mask_channel=True)

In [17]:
obs = env.reset()

In [18]:
obs = np.array([obs]) #Convierto en un batch de 1 (1 env paralelo)

In [19]:
obs[0][0] #Tablero inicial

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0, -1, -1,  0,  0,  0],
       [ 0,  0,  0,  1, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [20]:
# Testeo de forward
model.model.policy(th.from_numpy(obs).to(model.model.device))

(tensor([19], device='cuda:0'),
 tensor([[-0.4992]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([-1.0972], device='cuda:0', dtype=torch.float64,
        grad_fn=<SqueezeBackward1>))

# Corremos PPO

In [21]:
model.learn(total_timesteps=100000)

Logging to tensorboard_log/PPO_65
Eval num_timesteps=8000, episode_reward=-0.10 +/- 0.96
Episode length: 30.01 +/- 0.71
---------------------------------
| eval/              |          |
|    mean_ep_length  | 30       |
|    mean_reward     | -0.1     |
| time/              |          |
|    total timesteps | 8000     |
---------------------------------
New best mean reward!
Eval num_timesteps=16000, episode_reward=-0.09 +/- 0.96
Episode length: 30.03 +/- 0.54
---------------------------------
| eval/              |          |
|    mean_ep_length  | 30       |
|    mean_reward     | -0.09    |
| time/              |          |
|    total timesteps | 16000    |
---------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29.7     |
|    ep_rew_mean     | 0.18     |
| time/              |          |
|    fps             | 300      |
|    iterations      | 1        |
|    time_elapsed    | 54       |
|

#### Aclaración para docentes: Para mejores corridas referirse a la notebook 009_Analisis donde muestro todos los resultados de los diferentes experimentos